### Experiment : Set up optuna with neptune for Hyper parameter and architecture buildinng
### Objectives

1. batch size =[8,16,32]
2. learining rate = [0.1,0.01,0.0001,0.00001]
3. units = [8,16,32,64,128]
4. number of layers in architecture [4,5,6]
5. optimization = [adam,]
6. epochs = [10,20]

### Observations

1. 
2. 
3. 

### Solutions and further investigation

1. 
2. 
3. 

### Load Modeules and datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV,train_test_split
from keras import layers
from sklearn import metrics
from keras import regularizers
from keras import models,optimizers
from keras.models import Sequential
from tensorflow.python.keras.optimizer_v2.adam import Adam
from keras.layers import Dense,Input,SpatialDropout1D,Dropout,Flatten, SimpleRNN,LSTM,RNN,GRU
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping
import re
import gensim
pd.set_option('display.max_colwidth', 1000)
# import helper function script
import sys
sys.path.insert(1,'G:\\Github\\Sinhala-Hate-Speech-Detection')
import utills
import sinhala_stemmer

In [ ]:
from neptune.new.integrations.tensorflow_keras import NeptuneCallback
#neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
#run.stop()
#run = neptune.init(project_qualified_name= NEPTUNE_PROJECT,api_token=NEPTUNE_API_TOKEN) 

In [ ]:
import neptune
#import neptune.new as neptune
from neptunecontrib.monitoring.metrics import expand_prediction, log_class_metrics, log_binary_classification_metrics, log_classification_report,log_confusion_matrix,log_prediction_distribution
from neptunecontrib.api import log_table
import neptunecontrib.monitoring.optuna as opt_utils 
import os
from dotenv import load_dotenv

load_dotenv()
NEPTUNE_PROJECT= os.getenv('NEPTUNE_PROJECT')
NEPTUNE_API_TOKEN = os.getenv(('NEPTUNE_API_TOKEN'))
neptune.init(project_qualified_name= NEPTUNE_PROJECT,api_token=NEPTUNE_API_TOKEN) 
             

In [ ]:
# load datasets - no preprocessing
path = '../Datasets/processed/no_stemming/'
df_A = pd.read_csv(path+'df_A.csv')     

In [ ]:
# remove NaN entries in dataset
df_A.drop(index =[610,3070],inplace=True)

### Embedding

In [ ]:
def load_emb_model(emb_model):
    from gensim.models import word2vec
    import gensim.models as FastText
    from gensim.test.utils import datapath
    
    if emb_model == "w2v_skipgram":
        model = word2vec.Word2Vec.load("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/word2vec/word2vec_300.w2v")
    elif emb_model == "w2v_cbow":
        model = word2vec.Word2Vec.load("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/CBOW-word2vec/cbow_300.w2v")
    elif emb_model == "fasttext":
        #FastText.load_fasttext_format("../../../corpus/analyzed/saved_models/wiki.si.bin")
        model = FastText.fasttext.load_facebook_model(datapath("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/cc.si.300.bin"))
        #model = word2vec.Word2Vec.load("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/fasttext_300.w2v")
    else:
        print("Invalid argument. Need w2v_skipgram or w2v_cbow or fasttext as argument")
        model = None
    return model
        

In [ ]:
def get_emb_index(model):
    from gensim.models import word2vec
    embeddings_index ={}
    for index, word in enumerate(model.wv.index_to_key):
        embeddings_index[word] = model.wv.get_vector(word)
    print('found %s word vectors' % len(embeddings_index))
    return embeddings_index

In [ ]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
def fasttext_OOV(model,oov_vocab,embed_index):
    count = 0
    for word, freq in oov_vocab:
        if word in model.wv:
            embed_index[word] = model.wv.get_vector(word)
            count+=1
    print(f"Added {count} words to embedding with rare words handling of fasttext")
    return embed_index

In [ ]:
model_fasttext = load_emb_model("fasttext")

In [ ]:
vocab = build_vocab(df_A['cleaned'])
embed_fasttext = get_emb_index(model_fasttext)
oov_fasttext = check_coverage(vocab, embed_fasttext)
emb_index_fasttext =fasttext_OOV(model_fasttext,oov_fasttext,embed_fasttext)
check_coverage(vocab, emb_index_fasttext)

In [ ]:
LEN_VOCAB = 20000

In [ ]:
MAX_SEQ_LEN = 100
def max_len(df):
  train_df = pd.DataFrame()
  train_df['doc_len'] = df.apply(lambda words: len(words.split()))
  MAX_SEQ_LEN = max(train_df['doc_len'])
  #MAX_SEQ_LEN = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
  print("max seq len",MAX_SEQ_LEN)
  return MAX_SEQ_LEN

# create a tokenizer 
def create_sequence(X_train,X_test,LEN_VOCAB):

  token = Tokenizer(num_words=LEN_VOCAB)
  token.fit_on_texts(X_train)
  word_index = token.word_index
  print("dictionary size: ", len(word_index))

  # ensure equal length vectors 
  train_seq_x = sequence.pad_sequences(token.texts_to_sequences(X_train), maxlen=MAX_SEQ_LEN)
  test_seq_x = sequence.pad_sequences(token.texts_to_sequences(X_test), maxlen=MAX_SEQ_LEN)
  return (train_seq_x,test_seq_x,word_index)

# create token-embedding mapping for 
def word_mapping(word_index):
    words_not_found=[]
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    # embedding_matrix = np.random.uniform(-0.25, 0.25, size=(vocab, self.EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: #and len(embedding_vector) > 0
            embedding_matrix[i] = embedding_vector
        else:
            embedding_matrix[i] = np.random.randn(300)
            words_not_found.append(word)
    print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    #print("sample words not found: ", np.random.choice(words_not_found, 200))
    print("Number of words not found",len(words_not_found))
    print("Propotion of oov ",(len(words_not_found)/len(word_index))*100)
    return embedding_matrix


In [ ]:
def embedding_matrix(embeddings_index, word_index, len_voc):
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = word_index
    embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    
    for word, i in word_index.items():
        if i >= len_voc:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

### Model Building

In [ ]:
def train_model(model,train_seq,test_seq,Y_train,Y_test,tag):
  
  #define callbacks
  early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=1,mode='min',restore_best_weights=True)
  #callbacks_list = [early_stopping]

  #model training
  print("started training")
  hist = model.fit(train_seq, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping],validation_split=0.1, shuffle=False, verbose=2)
  train_loss, train_acc = model.evaluate(train_seq, Y_train,batch_size=BATCH_SIZE, verbose=1)
  print("train loss - ",train_loss," train acc- ",train_acc)
  test_loss, test_acc = model.evaluate(test_seq,Y_test,batch_size=BATCH_SIZE, verbose=1)
  print("test loss - ",test_loss," test acc- ",test_acc)

  # plot loss during training
  #from matplotlib import pyplot
  print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
  plt.subplot(211)
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.ylabel("Cross Entropy Loss")
  plt.plot(hist.history['loss'],lw=2.0, color='b', label='train')
  plt.plot(hist.history['val_loss'],lw=2.0, color='r', label='validation')
  plt.legend(loc='upper right')
  # plot accuracy during training
  plt.subplot(212)
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.plot(hist.history['accuracy'],lw=2.0, color='b', label='train')
  plt.plot(hist.history['val_accuracy'],lw=2.0, color='r', label='validation')
  plt.legend(loc='upper right')
  plt.show()
  #(model.predict(x) > 0.5).astype("int32")
  #np.where(y_pred > threshold, 1,0)
  Y_pred =(model.predict(test_seq) > 0.5).astype("int32")# model.predict_class(test_seq)
  Y_pred = np.squeeze(Y_pred)
  log_result(Y_test,Y_pred,"fasttext","RNN",tag)
  print(classification_report(Y_test,Y_pred))
  #Y_pred = np.squeeze(Y_pred)
  utills.confusion_Matrix(Y_test,Y_pred)
  utills.PlotRocAuc(Y_test,Y_pred,'blue',"RNN")
  return model,Y_pred

In [ ]:
def log_result(Y_test,Y_pred,feature_name,model_name,tag):
    print("========= Eperiment - ",tag," =========")
    neptune.create_experiment(tag,params=PARAMS)
    neptune.append_tag(['DL experiment',feature_name,model_name,tag])
    
    log_class_metrics(Y_test, Y_pred)
    log_confusion_matrix(Y_test, Y_pred)
    log_classification_report(Y_test, Y_pred)

In [ ]:
model = Sequential()
model.add(Embedding(output_dim=EMBEDDING_DIM, 
                    input_dim=LEN_VOCAB, 
                    input_length=MAX_SEQ_LEN,
                    weights=[emb_matrix], # Additionally we give the Wi
                    trainable=False)) # Don't train the embeddings - just use GloVe embeddings
# We can start with pre-trained embeddings and then fine-tune them using our data by setting trainable to True
model.add(SimpleRNN(128, activation='relu',dropout=0.2, recurrent_dropout=0.3))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
optimizer_adam = Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer_adam,
              metrics=['accuracy'])
model.summary()

In [ ]:
def create_model(trial,emb_matrix):
    layers = 4
    units = [8,16,32,64,128,300]
    unit = trial.suggest_int("units",units)
    model = Sequential()
    model.add(Embedding(output_dim=EMBEDDING_DIM, 
                        input_dim=LEN_VOCAB, 
                        input_length=MAX_SEQ_LEN,
                        weights=[emb_matrix], # Additionally we give the Wi
                        trainable=False)) # Don't train the embeddings - just use GloVe embeddings
    # We can start with pre-trained embeddings and then fine-tune them using our data by setting trainable to True

    model.add(SimpleRNN(unit, activation='relu',dropout=0.2, recurrent_dropout=0.3))
    dropout_rate = trial.suggest_float("dropout",0.0,0.2,0.3)
    #activation = trial.suggest_categorical("activation", ["relu", "selu", "elu", "swish"])
    model.add(Dropout(dropout_rate))
    model.add(Dense(unit, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation='sigmoid'))
    optimizer_adam = Adam(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy',
                optimizer=optimizer_adam,
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
PARAMS = {'epoch': EPOCHS,
          'lr': 0.0001,
          'batch':BATCH_SIZE,
          'stemmed':False,
          'embedding':'fasttext',
          'emb_trainable':False
          }
RNN_model,prediction = train_model(model,X_train,X_test,Y_train,Y_test,"SimpleRNN + fasttext")

### Optuna set up

In [ ]:
import optuna
from optuna import trial
def objective(trial):  # trial is object
    model = create_model(trial)
    scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/BATCH_SIZE), 1e-5)
    lr = LearningRateScheduler(scheduler, verbose=0)
    
        # Fit the model on the training data.
        # The TFKerasPruningCallback checks for pruning condition every epoch.
    model.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        callbacks=[TFKerasPruningCallback(trial, "val_loss")],
        epochs=EPOCHS,
        validation_data=(X_test, y_test),
        verbose=1,
    )
    f1_score = model_performence(model)
    return f1_score
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective,n_trials=N_TRIALS)  # number of trials

In [ ]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=100)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_param_importances(study)

In [ ]:
#  lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
# #         lr = WarmupExponentialDecay(lr_base=1e-3, decay=1e-5, warmup_epochs=30)
#         es = EarlyStopping(monitor="val_loss", patience=60, verbose=1, mode="min", restore_best_weights=True)
    
#         checkpoint_filepath = f"folds{fold}.hdf5"
#         sv = keras.callbacks.ModelCheckpoint(
#             checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
#             save_weights_only=False, mode='auto', save_freq='epoch',
#             options=None
#         )

#         model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])
#         #model.save(f'Fold{fold+1} RNN Weights')
#         test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

In [ ]:
import glob

model.save('my_model')

run_2['my_model/saved_model'].upload('my_model/saved_model.pb')

for name in glob.glob('my_model/variables/*'):
    run_2[name].upload(name)

In [ ]:
# Log single String
run['aux/text'] = 'text I keep track of, like query or tokenized word'

# Log series of String to one log
for epoch in range(epochs_nr):
    token = str(...)
    run['train/tokens'].log(token)
## CSV file
#run['test/preds'].upload('path/to/test_preds.csv')
from neptune.new.types import File

# Pandas DataFrame 
#run['data/sample'].upload(File.as_html(sample_df))